**Extract all options and collect all their values across the commit history**

Problems (TODO)
- options in config files that appear multiple time, such as COPY/ADD/RUN/FROM in Dockerfile
- there is no way to reliably track each option seperately
- therefore we currently exclude such options

Definition of columns
- `Changed internally` is an integer, indicating how often the value of an option was changes in the project
- `Removed` is a boolean, indicating if an option has been removed at some point 

In [1]:
import pandas as pd
from typing import List, Tuple


def extract_options(data: List) -> Tuple:
    """
    Extract all options and all of their values from the commit history of a software projects.

    :param data: list of configuration data from commit history
    :return: tuple of dataframes containing the results and excluded options
    """

    project_name = data["project_name"]
    print(f"Extract all options and their values from {project_name}.")

    # Extract configuration options and their values, excluding duplicates
    config_data = []
    excluded_pairs = set()
    option_presence_tracker = {}  # Track presence across commits

    for commit in data["config_commit_data"]:
        if commit["is_config_related"]:
            commit_hash = commit["commit_hash"]
            for file_data in commit["network_data"]["config_files_data"]:
                # Dictionary to track option occurrences in the current file
                option_tracker = {}
                for pair in file_data["pairs"]:
                    key = (file_data["file_path"], pair["option"])
                    
                    if key not in option_tracker:
                        option_tracker[key] = []
                    option_tracker[key].append(pair)
                    
                    # Update the presence tracker
                    if key not in option_presence_tracker:
                        option_presence_tracker[key] = {"last_seen": commit_hash, "removed": False}
                    else:
                        option_presence_tracker[key]["last_seen"] = commit_hash
                        option_presence_tracker[key]["removed"] = False  # Mark as seen in this commit

                # Add only options that appear once in the file
                for key, occurrences in option_tracker.items():
                    if len(occurrences) == 1:  # Include only unique options
                        pair = occurrences[0]
                        config_data.append({
                            "file_path": file_data["file_path"],
                            "option": pair["option"],
                            "value": pair["value"],
                            "type": pair["type"],
                            "concept": file_data["concept"]
                        })
                    else:
                        pair = occurrences[0]
                        excluded_pairs.add((file_data["file_path"], pair["option"], file_data["concept"]))

    # After processing all commits, check for removed options
    for key, data in option_presence_tracker.items():
        if data["last_seen"] != commit_hash:  # If not seen in the last commit, mark as removed
            option_presence_tracker[key]["removed"] = True

    # Create DataFrame from the extracted data
    df = pd.DataFrame(config_data)

    df_excluded = pd.DataFrame(list(excluded_pairs))

    # store excludes options only if dataframe is not empty
    if not df_excluded.empty:
        df_excluded.columns = ["File", "Option", "Concept"]

    # Group by option, type, and file_path, and aggregate unique values
    aggregated_df = (
        df.groupby(['file_path', 'option', 'concept'])['value']
        .apply(lambda x: sorted(list(set(x))))
        .reset_index()
    )

    # Rename columns for clarity
    aggregated_df.columns = ['File Path', 'Option', 'Concept', 'Values']

    # Add and 'changed internally' columns
    aggregated_df['Changed internally'] = aggregated_df['Values'].apply(lambda x: len(x) - 1 if len(x) > 1 else 0)

    # Add 'removed' column by checking the option presence tracker
    removed_status = []
    for _, row in aggregated_df.iterrows():
        key = (row['File Path'], row['Option'])
        removed_status.append(option_presence_tracker.get(key, {}).get('removed', False))

    aggregated_df['Removed'] = removed_status

    return aggregated_df, df_excluded

In [5]:
import glob
import json

analyzed_project_dir = "../data/analyzed_projects"

for project_path in glob.glob(analyzed_project_dir + "/**"):
    with open(project_path, "r", encoding="utf-8") as src:
        data = json.load(src)
        project_name = data["project_name"]

        df_result, df_excluded = extract_options(data=data)

        df_excluded.to_csv(f"../data/excluded_options/{project_name}_excluded.csv", index=False)
        df_result.to_csv(f"../data/extracted_options/{project_name}_options.csv", index=False)

Extract all options and their values from tmpjy99xo7_.
Extract all options and their values from tmpvrgbmja6.
Extract all options and their values from tmpuvyqome2.
Extract all options and their values from tmprubhzu3f.
Extract all options and their values from tmp3c5ymdly.
Extract all options and their values from tmpji17hn9k.
Extract all options and their values from tmpg6do6nsj.
Extract all options and their values from tmpu3uu352p.


**Extract if an options was set in other projects if the option was changed**

Definitions of columns
- `Changed globally` is an integer, indicating if an option was changed in other projects
- `Set globally` is an integer, indicating the number of projects in which the option exists
- `Occurrences globally` is an integer, indicating how often the option occurs across all projects

In [4]:
import os
import ast
import pandas as pd


def analyze_options(target_df, other_dfs) -> pd.DataFrame:
    """
    Analyze options in a target file against all other files to compute global stats.

    :param target_df: dataframe of target project
    :param other_dfs: dataframes of all other projects
    :return target_df: updated dataframe of target project
    """
    # Initialize columns
    target_df['Set globally'] = 0
    target_df['Changed globally'] = 0
    target_df['Occurrences globally'] = 0


    for index, row in target_df.iterrows():
        option = row['Option']

        for other_df in other_dfs:
            # Find all rows in other_df where the option matches
            matching_rows = other_df[other_df['Option'] == option]
            match_count = len(matching_rows)

            if match_count > 0:
                # Increment "Set in other projects" by 1 (project-level count)
                target_df.loc[index, 'Set globally'] += 1

                # Increment "Total occurrences" by the total count of matches
                target_df.loc[index, 'Occurrences globally'] += match_count

                # Check each match for changes in values
                for _, match_row in matching_rows.iterrows():
                    # Parse the 'Values' column (convert from string to list if necessary)
                    raw_values = match_row['Values']
                    try:
                        values = ast.literal_eval(raw_values) if isinstance(raw_values, str) else raw_values
                    except (ValueError, SyntaxError):
                        values = [raw_values]  # Fall back to treating as a single value

                    # Ensure `values` is iterable
                    if not isinstance(values, (list, set, tuple)):
                        values = [values]

                    unique_values = set(values)
                    if len(unique_values) > 1:
                        # Increment "Changed globally" for each such occurrence
                        target_df.loc[index, 'Changed globally'] += 1

    return target_df


data_dir = "../data/test_data/extracted_options"

# Load all CSV files from the directory into a dictionary of DataFrames
repository_files = [file for file in os.listdir(data_dir) if file.endswith('.csv')]
repository_dataframes = {file: pd.read_csv(os.path.join(data_dir, file)) for file in repository_files}

target_file_name = 'piggymetrics_options.csv'
target_df = repository_dataframes[target_file_name]

# Use all other files as comparison
other_dfs = [df for name, df in repository_dataframes.items() if name != target_file_name]

# Perform the analysis
updated_target_df = analyze_options(target_df.copy(), other_dfs)

updated_target_df.head(50)


,File Path,Option,Concept,Values,Changed internally,Removed,Set globally,Changed globally,Occurrences globally
0,.travis.yml,after_success,travis,"['bash <(curl -s https://codecov.io/bash)', 'c...",1,False,0,0,0
1,.travis.yml,before_install,travis,"['pip install codecov', 'sudo pip install code...",1,True,0,0,0
2,.travis.yml,dist,travis,['trusty'],0,False,0,0,0
3,.travis.yml,env.global,travis,"['COMMIT=${TRAVIS_COMMIT::7}', 'COMMIT=${TRAVI...",1,False,0,0,0
4,.travis.yml,env.global.secure/GeONVsTD48Y88CKoqupo/FC1Gy0e...,travis,['GeONVsTD48Y88CKoqupo/FC1Gy0eCrT1UNylvMzz5VYc...,0,False,0,0,0
5,.travis.yml,env.global.secure/Gl6a03cI88dKHV4rjP1IkYqCdVe7...,travis,['Gl6a03cI88dKHV4rjP1IkYqCdVe7IM0XNcEzFDCxmvXH...,0,False,0,0,0
6,.travis.yml,env.global.secure/Kt8hRbEIkYW3CYJgvqEK9zCyS/l5...,travis,['Kt8hRbEIkYW3CYJgvqEK9zCyS/l54XOI90799xEuXWgF...,0,True,0,0,0
7,.travis.yml,env.global.secure/VRlJyPOz7fUmtFdpTdO51BVcjKUG...,travis,['VRlJyPOz7fUmtFdpTdO51BVcjKUGP5t7KF5bG7TSJPXs...,0,False,0,0,0
8,.travis.yml,jdk,travis,['oraclejdk8'],0,False,0,0,0
9,.travis.yml,language,travis,['java'],0,False,0,0,0


In [ ]:
import pandas as pd
import ipytest
import json

ipytest.autoconfig()


def test_extract_options():
    
    # Load commit history data
    with open("../data/test_data/projectB_data.json", "r", encoding="utf-8") as src:
        data = json.load(src)

    # Extract options
    df_results, df_excluded = extract_options(data=data)

    # TODO

def test_analyze_options():
    # Create target_df
    target_df = pd.read_csv("../data/test_data/projectA_options.csv")

    # Create other_dfs
    other_df1 = pd.read_csv("../data/test_data/projectB_options.csv")
    other_df2 = pd.read_csv("../data/test_data/projectC_options.csv")
    other_dfs = [other_df1, other_df2]

    # Analyze configurations
    result_df = analyze_options(target_df, other_dfs)


    print(result_df.head())

    assert result_df.loc[result_df['Option'] == 'EXPOSE', 'Set globally'].iloc[0] == 2
    assert result_df.loc[result_df['Option'] == 'EXPOSE', 'Occurrences globally'].iloc[0] == 11
    assert result_df.loc[result_df['Option'] == 'EXPOSE', 'Changed globally'].iloc[0] == 3

    assert result_df.loc[result_df['Option'] == 'project.version', 'Set globally'].iloc[0] == 2
    assert result_df.loc[result_df['Option'] == 'project.version', 'Occurrences globally'].iloc[0] == 11
    assert result_df.loc[result_df['Option'] == 'project.version', 'Changed globally'].iloc[0] == 0

    assert result_df.loc[result_df['Option'] == 'server.port', 'Set globally'].iloc[0] == 2
    assert result_df.loc[result_df['Option'] == 'server.port', 'Occurrences globally'].iloc[0] == 14
    assert result_df.loc[result_df['Option'] == 'server.port', 'Changed globally'].iloc[0] == 7

ipytest.run("-vv")